In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer,DataCollatorWithPadding

In [2]:
raw_datasets = load_dataset('glue','mrpc')

In [3]:
checkpoint = 'bert-base-uncased'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [5]:
def tokenize_function(examples):
    return tokenizer(examples['sentence1'],examples['sentence2'],truncation=True)

In [6]:
tokenized_datasets = raw_datasets.map(tokenize_function,batched=True)

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [9]:
train_dataset = tokenized_datasets['train']
eval_dataset = tokenized_datasets['validation']
train_dataset.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [10]:
from torch.utils.data import DataLoader

In [11]:
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

In [12]:
eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=8,
    collate_fn=data_collator,
)

In [13]:
for batch in train_dataloader:
    break
{k:v.shape for k,v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 65]),
 'token_type_ids': torch.Size([8, 65]),
 'attention_mask': torch.Size([8, 65])}

In [14]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
train_dataset[1]['labels']

tensor(0)

In [16]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [17]:
output = model(**batch)
output.loss,output.logits.shape

(tensor(0.5668, grad_fn=<NllLossBackward0>), torch.Size([8, 2]))

In [18]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(),lr=5e-5)
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [19]:
lr_scheduler

In [20]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [21]:
from tqdm.auto import tqdm

In [22]:
progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/1377 [00:00<?, ?it/s]

In [27]:
model.to(device)
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


In [28]:
import evaluate
metric = evaluate.load('glue','mrpc')
model.eval()
for batch in eval_dataloader:
    batch = {k:v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits,dim=-1)
    metric.add_batch(predictions=predictions,references=batch['labels'])
metric.compute()

{'accuracy': 0.8480392156862745, 'f1': 0.8959731543624161}